In [1]:
import pykep as pk
import numpy as np
import json
import pickle as pkl

from mpl_toolkits.mplot3d import Axes3D
from matplotlib import pyplot as plt
%matplotlib notebook

In [2]:
import cascade as csc
from copy import deepcopy
from tqdm.notebook import tqdm
import heyoka as hy

# We import the simulation initial conditions and atmospheric density model
The files needed are:
* **debris_simulation_ic.pk** - created by the notebook 1
* **best_fit_density.pk** - created by the notebook 1b

In [29]:
with open("data/debris_simulation_ic.pk", "rb") as file:
    r_ic,v_ic,c_radius,to_satcat,satcat,debris = pkl.load(file)

* **r**: contains the initial position of all satellites to be simulated (SI units)
* **v**: contains the initial velocity of all satellites to be simulated (SI units)
* **radius**: contains all the estimated radii for the various objects (in meters)
* **to_satcat**: contains the indexes in the satcat of the corresponding r,v,radius entry
* **satcat**: the satcat
* **debris**: the corresponding pykep planets

In [30]:
with open("data/best_fit_density.pk", "rb") as file:
    best_x = pkl.load(file)

In [31]:
# We need to create an array containing all B*
BSTARS = []
for idx in to_satcat:
    BSTARS.append(float(satcat[idx]["BSTAR"]))
BSTARS = np.array(BSTARS)
# We remove negative BSTARS setting the value to zero in tbose occasions
BSTARS[BSTARS<0] = 0.

# We build the dynamical system to integrate

In [32]:
# This little helper returns the heyoka expression for the density using
# the results from the data interpolation
def compute_density(h, best_x):
    """
    returns the heyoka expression for the atmosheric density in kg.m^3. Input is the altitude in m. (when we fitted
    km were used here we change as to allow better expressions)
    """
    p1 = np.array(best_x[:4])
    p2 = np.array(best_x[4:8]) / 1000
    p3 = np.array(best_x[8:]) * 1000
    retval = 0.
    for alpha,beta, gamma in zip(p1,p2, p3):
        retval += alpha*hy.exp(-(h-gamma)*beta)
    return retval

In [33]:
# Dynamical variables.
x,y,z,vx,vy,vz = hy.make_vars("x","y","z","vx","vy","vz")

# Constants.
GMe = pk.MU_EARTH
C20 = -4.84165371736e-4
Re = pk.EARTH_RADIUS

# Create Keplerian dynamics.
dyn = csc.dynamics.kepler(mu = GMe)

# Add the J2 terms.
magR2 = hy.sum_sq([x, y, z])
J2term1 = GMe*(Re**2)*np.sqrt(5)*C20/(2*magR2**(1./2))
J2term2 = 3/(magR2**2)
J2term3 = 15*(z**2)/(magR2**3)
fJ2X = J2term1*x*(J2term2 - J2term3)
fJ2Y = J2term1*y*(J2term2 - J2term3)
fJ2Z = J2term1*z*(3*J2term2 - J2term3)
dyn[3] = (dyn[3][0], dyn[3][1] + fJ2X)
dyn[4] = (dyn[4][0], dyn[4][1] + fJ2Y)
dyn[5] = (dyn[5][0], dyn[5][1] + fJ2Z)

# Adds the drag force.
magV2 = hy.sum_sq([vx, vy, vz])
magV = hy.sqrt(magV2)
# Here we consider a spherical Earth ... easy to account for the oblateness effect.
altitude = (hy.sqrt(magR2) - Re)
density = compute_density(altitude, best_x)
ref_density = 0.1570 / pk.EARTH_RADIUS
fdrag = density / ref_density * hy.par[0] * magV
fdragX = - fdrag * vx
fdragY = - fdrag * vy
fdragZ = - fdrag * vz
dyn[3] = (dyn[3][0], dyn[3][1] + fdragX)
dyn[4] = (dyn[4][0], dyn[4][1] + fdragY)
dyn[5] = (dyn[5][0], dyn[5][1] + fdragZ)

In [52]:
sim = csc.sim(r_ic[:,0],r_ic[:,1],r_ic[:,2],v_ic[:,0],v_ic[:,1],v_ic[:,2],c_radius,0.23 * 806.81,dyn=dyn,pars=[BSTARS], c_radius=pk.EARTH_RADIUS+78000.)

In [53]:
r_ic = np.delete(r_ic, 18931, axis=0)
BSTARS = np.delete(BSTARS, 18931, axis=0)
v_ic = np.delete(v_ic, 18931, axis=0)
to_satcat = np.delete(to_satcat, 18931, axis=0)
c_radius = np.delete(c_radius, 18931, axis=0)

In [55]:
final_t = 365.25 * pk.DAY2SEC

pbar = tqdm(total=final_t)

while sim.time < final_t:
    orig_time = sim.time
    
    oc = sim.step()
    
    pbar.update(sim.time - orig_time)
   
    if oc == csc.outcome.collision:
        # TODO different code needed for crash
        # on Earth here.
        pi, pj = sim.interrupt_info
        
        print("Collision detected, re-initing particles {} and {}".format(pi, pj))
        
        new_x = deepcopy(sim.x)
        new_y = deepcopy(sim.y)
        new_z = deepcopy(sim.z)
        
        new_vx = deepcopy(sim.vx)
        new_vy = deepcopy(sim.vy)
        new_vz = deepcopy(sim.vz)
        
        new_sizes = deepcopy(sim.sizes)

        for idx in [pi, pj]:
            a = rng.uniform(1.02*Re, 1.3*Re)
            e = rng.uniform(0, 0.02)
            inc = rng.uniform(0, 0.05)
            om = rng.uniform(0, 2*np.pi)
            Om = rng.uniform(0, 2*np.pi)
            nu = rng.uniform(0, 2*np.pi)
            size = rng.uniform(0.01, 0.1)

            r, v = pk.par2ic([a, e, inc, om, Om, nu], GMe)

            new_x[idx] = r[0]
            new_y[idx] = r[1]
            new_z[idx] = r[2]

            new_vx[idx] = v[0]
            new_vy[idx] = v[1]
            new_vz[idx] = v[2]
            
            new_sizes[idx] = size

        sim.set_new_state(new_x, new_y, new_z, new_vx, new_vy, new_vz, new_sizes)
    elif oc == csc.outcome.reentry:
        pi = sim.interrupt_info
        
        print("Particle {} re-entered".format(pi))
        break

pbar.close()
del pbar

  0%|          | 0/31557600.0 [00:00<?, ?it/s]

Particle 17422 re-entered
